<a href="https://colab.research.google.com/github/Mammad-AI/IAAA_MMD_AI/blob/main/first_densenet_for_iaaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the necessary packages for working with DICOM files and MONAI
!pip install pydicom
!pip install monai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.0 MB/s eta 0:00:00


In [2]:
# Uploading files from the user's local system
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohammadpourmazahery","key":"6fea11414884209611b85dbd61351ce5"}'}

In [3]:
from monai.transforms import (
    Compose, LoadImage, EnsureChannelFirstD, ScaleIntensity, Resize, ToTensor,
    RandFlip, RandRotate90, RandScaleIntensity, RandShiftIntensity
)
from monai.networks.nets import UNet, UNETR, SegResNet
from monai.data import DataLoader, Dataset
from monai.inferers import sliding_window_inference

In [4]:
import monai
from monai.apps import download_and_extract

In [5]:
# Importing the necessary libraries for data processing and model building
import random
import pandas as pd  # For handling CSV files and dataframes
import os  # For interacting with the file system
import pydicom  # For working with DICOM files
import matplotlib.pyplot as plt  # For visualizations
import seaborn as sns  # For enhanced data visualizations
from tqdm import tqdm  # For progress bars in loops
import pydicom as dicom  # Alias for pydicom
from pydicom import dcmread  # To read DICOM files
from pydicom.data import get_testdata_file  # To access sample DICOM data
from pydicom.pixel_data_handlers.util import apply_voi_lut  # Utility to handle DICOM pixel data transformations
import torchvision.models as models  # For pre-trained models
import numpy as np  # For array manipulations
from sklearn.metrics import accuracy_score  # For evaluating classification models
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # For image preprocessing in Keras
from natsort import natsorted  # For natural sorting
import cv2  # For image processing
import math  # For mathematical operations
from torch.utils.data import DataLoader, Dataset  # For creating datasets and loading data in PyTorch
import torch.nn.functional as F  # For using various functions in PyTorch
import torch.nn as nn  # For building neural networks
import torch.optim as optim  # For optimization algorithms
from sklearn.model_selection import KFold  # For K-Fold cross-validation
import sys  # For system-related functions
import torch  # PyTorch library


from monai.networks.nets import DenseNet121  # For using DenseNet architecture
from monai.data import Dataset, CacheDataset  # For handling medical datasets
from monai.utils import set_determinism  # For reproducibility in training


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# Set up Kaggle API credentials for downloading the dataset
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions for the Kaggle JSON file



In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [71]:
model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2)
model.features.transition3.pool=nn.AvgPool3d(kernel_size=(1,2,2), stride=(1,2,2))
model.class_layers.out = nn.Sequential(
    nn.Linear(1024, 2),  # Replace 1024 with the actual input size of your fc layer
    nn.Softmax(dim=1)  # Apply softmax activation
)
model.to(device)

DenseNet121(
  (features): Sequential(
    (conv0): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
      (denselayer2): _DenseLayer(
        (layers): Sequential(
 

In [11]:
# Download and unzip the MRI dataset from Kaggle
!kaggle datasets download -d iaaaplatform/iaaa-mri-challenge
!unzip iaaa-mri-challenge.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/1.3.46.670589.11.10042.5.0.7984.2024022407234101045/1.3.46.670589.11.10042.5.0.7984.2024022407242432073.dcm  
  inflating: data/1.3.46.670589.11.10042.5.0.7984.2024022407234101045/1.3.46.670589.11.10042.5.0.7984.2024022407242437074.dcm  
  inflating: data/1.3.46.670589.11.10042.5.0.7984.2024022407234101045/1.3.46.670589.11.10042.5.0.7984.2024022407242443075.dcm  
  inflating: data/1.3.46.670589.11.10042.5.0.7984.2024022407234101045/1.3.46.670589.11.10042.5.0.7984.2024022407242448076.dcm  
  inflating: data/1.3.46.670589.11.10042.5.0.7984.2024022407234101045/1.3.46.670589.11.10042.5.0.7984.2024022407242453077.dcm  
  inflating: data/1.3.46.670589.11.10042.5.0.7984.2024022407234101045/1.3.46.670589.11.10042.5.0.7984.2024022407245301078.dcm  
  inflating: data/1.3.46.670589.11.10042.5.0.7984.2024022407234101045/1.3.46.670589.11.10042.5.0.7984.2024022407245306079.dcm  
  inflating: data/1.3.46.670589.11.10042.5.0.7984.202

In [12]:
# Define paths and load the training labels from the CSV file
data_dir = 'data'
train_label = pd.read_csv('train.csv')



In [13]:
# Function to sort DICOM files by slice number based on a specific DICOM tag
def sort_dicom_files(paths):
    sorted_files = [np.nan] * 24  # Initialize a list to store sorted files
    for i in range(len(paths)):
        dicom = pydicom.dcmread(paths[i])  # Read each DICOM file
        slice_number_mr_tag = (0x2001, 0x100a)  # DICOM tag for slice number
        slice_number_mr_value = int(dicom[slice_number_mr_tag].value)  # Get the slice number
        sorted_files[(slice_number_mr_value - 1)] = paths[i]  # Place the file in the correct position
    filtered_paths = [path for path in sorted_files if not isinstance(path, float)]  # Remove empty slots
    return filtered_paths  # Return the sorted list of paths


In [14]:
# Function to generate labels based on the `SeriesInstanceUID`
def produce_label(SeriesInstanceUID, train_label):
    label = train_label[train_label.SeriesInstanceUID == SeriesInstanceUID].prediction
    # If no label is found, default to [1,0]
    if label.empty:
        label = torch.tensor([1, 0])
    else:
        if label.iloc[0] == 0:
            label = torch.tensor([1, 0])
        else:
            label = torch.tensor([0, 1])
    return label



In [15]:
# Function to standardize DICOM files and resize them to a specific shape
def standardize_dicom(serie_id):
    path1 = f'data/{serie_id}'
    paths = []
    for i in os.listdir(path1):
        paths.append(os.path.join(path1, i))
    paths = sort_dicom_files(paths)  # Sort the DICOM files in the correct order
    dicom_3d = []
    for i in range(len(paths)):
        dir = paths[i]
        dicom_pix = pydicom.dcmread(dir).pixel_array  # Read the pixel data from the DICOM file
        dicom_pix = dicom_pix / dicom_pix.max()  # Normalize the pixel data
        dicom_3d.append(dicom_pix)  # Append the slice to the list
    dicom_3d = np.array(dicom_3d)  # Convert the list to a NumPy array
    dicom_3d = torch.tensor(dicom_3d)  # Convert the array to a PyTorch tensor
    dicom_3d = dicom_3d.unsqueeze(0)  # Add a channel dimension
    dicom_3d = dicom_3d.unsqueeze(0)  # Add a batch dimension
    dicom_3d = F.interpolate(dicom_3d, (16, 288, 288), mode='trilinear', align_corners=False)  # Resize the data to the required shape
    return dicom_3d  # Return the standardized DICOM data



In [16]:
# Custom dataset class for DICOM data
class DICOMDataset(Dataset):
    def __init__(self, series_list, produce_label, data_dir, transform=None):
        self.series_list = series_list  # List of series IDs
        self.produce_label = produce_label  # Function to produce labels
        self.data_dir = data_dir  # Directory containing the data
        self.transform = transform  # Transformations to be applied to the data

    def __len__(self):
        return len(self.series_list)  # Return the length of the dataset

    def __getitem__(self, idx):
        series_id = self.series_list[idx]  # Get the series ID for the current index
        final_input = standardize_dicom(series_id)  # Standardize the DICOM files for this series
        final_label = produce_label(series_id, train_label)  # Get the corresponding label
        final_input = final_input.clone().detach()  # Detach the tensor from the computation graph
        if self.transform:
            final_input = self.transform(final_input)  # Apply any transformations
        return final_input, final_label  # Return the input and label


In [17]:
# Freezing all model layers initially
for param in model.parameters():
    param.requires_grad = False


In [68]:

criterion = nn.CrossEntropyLoss()  # Mean Squared Error Loss
total_itteration=5
b=0
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001


In [19]:
def unfreeze_layers(model, num_layers_to_unfreeze, device):
    """
    Unfreeze the last `num_layers_to_unfreeze` layers of the model.
    Move the unfrozen layers to the specified device.
    """
    layers = list(model.children())
    for layer in layers[num_layers_to_unfreeze:]:
        for param in layer.parameters():
            param.requires_grad = True
        layer.to(device) # Move the unfrozen layer to the device

In [20]:
layers=list(model.features+model.class_layers)

In [21]:
layers_to_unfreeze=[-1,-6,-7,-8,-9,-10,-11,-12,-16]

In [22]:
epoch=len(layers_to_unfreeze)

In [23]:
abnormal_series_id=train_label[train_label.prediction==1].SeriesInstanceUID

In [24]:
normal_series_id=train_label[train_label.prediction==0].SeriesInstanceUID

In [51]:
series_list=np.concatenate((abnormal_series_id,np.random.choice(normal_series_id,391)))
random.shuffle(series_list)

In [26]:
kf=KFold(n_splits=3,shuffle=True,random_state=42)

In [ ]:
for i in range(total_itteration):
    series_list=np.concatenate((abnormal_series_id,np.random.choice(normal_series_id,391)))
    random.shuffle(series_list)
    num_epochs=len(layers_to_unfreeze)
    best_val_loss = float('inf')
    save_dir='saved_models'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it does not exist
    best_model_path = os.path.join(save_dir, 'best_model.pth')
    for train_index, test_index in kf.split(series_list):
        train_series_list, test_series_list = series_list[train_index], series_list[test_index]
        train_dataset = DICOMDataset(train_series_list, produce_label, data_dir)
        val_dataset = DICOMDataset(test_series_list, produce_label, data_dir)
        train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)
        # Training loop for this fold

        for epoch in range(len(layers_to_unfreeze)):
            for param in model.parameters():
                param.requires_grad = False

            unfreeze_layers(model, layers_to_unfreeze[epoch],device)  # Gradually unfreeze layers

        # Update the optimizer to include all trainable parameters
            optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=(0.001/(epoch+b+i)))
            model.train()
            running_loss=0.0
            for batch in tqdm(train_loader):
                inputs, labels = batch
                labels=labels.float()
                inputs = inputs.float()  # Convert inputs to float type
                inputs = inputs.squeeze(2) # Remove the extra dimension at index 2
                inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                loss.backward()
                optimizer.step()


            model.eval()
            torch.cuda.empty_cache()

            val_loss = 0.0
            with torch.no_grad():
                for batch in tqdm(val_loader):
                    inputs, labels = batch
                    inputs = inputs.float()  # Convert inputs to float type
                    inputs = inputs.to(device)
                    inputs=inputs.squeeze(2)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, labels.float())
                    val_loss += loss.item()
            torch.cuda.empty_cache()
            val_loss /= len(val_loader)
            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}')
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), best_model_path)
                print(f'Best model saved with validation loss: {best_val_loss}')
            print(outputs)
            print(labels)
        print(f'number of total kf={b}')
    print(f'number of total itteration={i}')
    best_model_weights = torch.load(best_model_path)
    model.load_state_dict(best_model_weights)
    print(i)


100%|██████████| 66/66 [00:13<00:00,  4.97it/s]


Epoch 1/9, Train Loss: 0.5218864187939476, Val Loss: 0.45103871867512213
Best model saved with validation loss: 0.45103871867512213
tensor([[0.2436, 0.7564]], device='cuda:0')
tensor([[1, 0]], device='cuda:0')


100%|██████████| 66/66 [00:13<00:00,  4.95it/s]


Epoch 2/9, Train Loss: 0.5096381998244133, Val Loss: 0.4388573363874898
Best model saved with validation loss: 0.4388573363874898
tensor([[0.9961, 0.0039]], device='cuda:0')
tensor([[1, 0]], device='cuda:0')


100%|██████████| 66/66 [00:13<00:00,  4.95it/s]


...

100%|██████████| 66/66 [00:13<00:00,  5.06it/s]


Epoch 5/9, Train Loss: 0.4096249844736725, Val Loss: 0.38679885232087335
tensor([[4.3300e-04, 9.9957e-01]], device='cuda:0')
tensor([[0, 1]], device='cuda:0')


100%|██████████| 66/66 [00:13<00:00,  4.98it/s]


Epoch 6/9, Train Loss: 0.4239022977479542, Val Loss: 0.38159057227048004
tensor([[0.2933, 0.7067]], device='cuda:0')
tensor([[0, 1]], device='cuda:0')


 37%|███▋      | 48/131 [00:09<00:13,  6.18it/s]

In [86]:
series_list=os.listdir('data')

In [94]:
with torch.no_grad():
    all_outputs=[]
    all_labels=[]
    for i in tqdm(series_list):
        input=standardize_dicom(i)
        input=input.float()
        input=input.to(device)
        output=model(input)
        output = output.cpu().detach().numpy()
        final_output=np.argmax(output)

        label=produce_label(i,train_label)
        label=label.cpu().detach().numpy()
        final_label=np.argmax(label)
        all_outputs.append(final_output)
        all_labels.append(final_label)


100%|██████████| 3132/3132 [09:30<00:00,  5.49it/s]


In [99]:
sum([o == l for o, l in zip(all_outputs, all_labels)]) / len(all_outputs)


0.8659003831417624

In [106]:

import pickle

# Save the lists to  Google Drive
with open('/content/drive/My Drive/all_labels.pkl', 'wb') as f:
  pickle.dump(all_labels, f)

with open('/content/drive/My Drive/all_outputs.pkl', 'wb') as f:
  pickle.dump(all_outputs, f)


In [107]:

from sklearn.metrics import f1_score

f1 = f1_score(all_labels, all_outputs, average='weighted')  # Calculate F1 score using 'weighted' average
print(f"F1 Score: {f1}")


F1 Score: 0.8836772546722171


In [93]:

from google.colab import files
import shutil

# Assuming 'best_model_path' variable holds the path to your best model
# Example: best_model_path = '/content/saved_models/best_model.pth'

# Define the destination path in your Google Drive
destination_path = '/content/drive/My Drive/best_model_dense_layer.pth'

# Copy the model file to your Google Drive
shutil.copy(best_model_path, destination_path)

print(f"Model weights saved to: {destination_path}")


Model weights saved to: /content/drive/My Drive/best_model_dense_layer.pth
